<a href="https://colab.research.google.com/github/SSR-ds/Code-Learnings/blob/master/Based_on_Grouping-%20Coding%20Denials.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib as mpl
import plotly.express as px
%matplotlib inline

import os
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from datetime import datetime
import seaborn as sns   

In [17]:
from google.colab import files
uploaded = files.upload()

Saving Denials_Check.xlsx to Denials_Check.xlsx


In [41]:
#df1 = pd.read_csv('Trusted Denied Lab Service Lines .csv',encoding='windows-1252')
df1 = pd.read_excel('Denials_Check.xlsx') 
df1.head()

,PayerName,PayeeName,PaymentDate,PaymentReferenceNumber,ClaimNumber,ClaimFilingIndicator,PatientLastName,PatientFirstname,PatientIDCode,ServiceDate,PayerClaimControlNumber,LineCount,ServiceRevenueCode,ServiceCode,ServiceChargeAmount,ServicePaymentAmount,PatientAmount,ServiceAllowable,Modifier1,Modifier2,Reason Codes
0,OXFORD HEALTH INSURANCE INC,THP DALLAS HILLCREST MC LLC,2020-02-27,9802187,HILLF7563-1,HM,Raghavendra,Sai,1296444603,2020-01-12,0030H02069,1,300,85027,207.16,306.7,1203.19,1509.89,25,NaN,[45] Charge exceeds fee schedule/maximum allow...
1,OXFORD HEALTH INSURANCE INC,THP DALLAS HILLCREST MC LLC,2020-02-27,9802187,HILLF7563-1,HM,Raghavendra,Sai,1296444603,2020-01-12,0030H02069,2,300,85009,245.21,0.0,NaN,0.00,25,NaN,[23] The impact of prior payer(s) adjudication...
2,OXFORD HEALTH INSURANCE INC,THP DALLAS HILLCREST MC LLC,2020-02-27,9802187,HILLF7563-1,HM,Raghavendra,Sai,1296444603,2020-01-12,0030H02069,3,300,87340,245.21,0.0,NaN,0.00,25,NaN,[23] The impact of prior payer(s) adjudication...
3,OXFORD HEALTH INSURANCE INC,THP DALLAS HILLCREST MC LLC,2020-02-28,9812345,HILLF8471-2,HM,Raghavendra,Sai,981140604,2020-01-23,0042H01324,1,300,87340,245.21,0.0,1551.15,1551.15,25,NaN,[45] Charge exceeds fee schedule/maximum allow...
4,OXFORD HEALTH INSURANCE INC,THP DALLAS HILLCREST MC LLC,2020-02-28,9812345,HILLF8471-2,HM,Raghavendra,Sai,981140604,2020-01-23,0042H01324,2,300,86803,245.21,0.0,NaN,0.00,25,NaN,[23] The impact of prior payer(s) adjudication...


In [42]:
df1['ServiceCode'] = df1['ServiceCode'].astype(str)
code_list = {"80047":["82330","82374","82435","82565","82947","84132","84295","84520"],
"80048":["82310","82374","82435","82565","82947","84132","84295","84520"],
"80050":["80053","85025","85027","85004","85027","85007","85009","84443"],
"80051":["82374","82435","84132","84295"],
"80053":["82040","82247","82310","82374","82435","82565","82947","84075","84132","84155","84295","84460","84450","84520"],
"80055":["85025","85027","85004","85027","85009","85007","87340","86762","86592","86850","86900","86901"],
"80081":["85025","85027","85004","85027","85007","85009","87340","87389","86762","86592","86850","86900","86901"],
"80061":["82465","83718","84478"],
"80069":["82040","82310","82374","82435","82565","82947","84100","84132","84295","84520"],
"80074":["86709","86705","87340","86803"],
"80076":["82040","82247","82248","84075","84155","84460","84450"]}

In [ ]:
#NR
new_dataframe = pd.DataFrame(columns=df1.columns)
new_dataframe['Mastercode'] = np.nan
new_dataframe['subcode'] = np.nan
patient_name_list = df1['PatientFirstname'] + ' ' + df1['PatientLastName']
patient_name_list = patient_name_list.unique()
for patient in patient_name_list:
  patient_details =  df1[(df1['PatientFirstname'] + ' ' + df1['PatientLastName'] ) == patient]
  patient_codes = patient_details['ServiceCode'].tolist()
  code_keys = code_list.keys()
  master_codes = []
  code_map = {}
  for code in patient_codes:
    if code in code_keys:
      master_codes.append(code)
  for master in master_codes:                                     
    patient_codes.remove(master)
    code_map[master] = master
  best_code, sub_list = bundle_code(code_list, patient_codes)
  while best_code != None:
    for sub in sub_list:
      patient_codes.remove(sub)
      code_map[sub] = best_code
    best_code, sub_list = bundle_code(code_list, patient_codes)
  for code in patient_codes:
    code_map[code] = code
  patient_details['Mastercode'], patient_details['subcode'] = zip(*patient_details['ServiceCode'].apply(check_master,code_list = code_list, priority_list = priList))
  new_dataframe = pd.concat([new_dataframe,patient_details])

In [43]:
def bundle_code(code_list, patient_codes):
  pri_master_dict = {}
  for master in code_list.keys():
    sub_code_list = code_list.get(master)
    sub_list = []
    for code in patient_codes:
      if code in sub_code_list:
        sub_list.append(code)
    pri_master_dict[master] = sub_list
  best_code = None
  bundle_size = 0
  for code in pri_master_dict.keys():
    if bundle_size < len(pri_master_dict.get(code)):
      bundle_size = len(pri_master_dict.get(code))
      best_code = code
  if bundle_size > 1:
    sub_list = pri_master_dict.get(best_code)
    return best_code, sub_list
  return None, None

In [44]:
def check_master(row, code_map):
  master_code = code_map.get(row)
  return master_code, row

In [45]:
patient_details =  df1
patient_codes = patient_details['ServiceCode'].tolist()
code_keys = code_list.keys()
code_map = {}
for code in patient_codes:
  if code in code_keys:
    patient_codes.remove(code)                                   
    code_map[code] = code
best_code, sub_list = bundle_code(code_list, patient_codes)
while best_code != None:
  for sub in sub_list:
    patient_codes.remove(sub)
    code_map[sub] = best_code
  best_code, sub_list = bundle_code(code_list, patient_codes)
for code in patient_codes:
  code_map[code] = code
patient_details['Mastercode'], patient_details['subcode'] = zip(*patient_details['ServiceCode'].apply(check_master,code_map = code_map))

In [47]:
patient_details

,PayerName,PayeeName,PaymentDate,PaymentReferenceNumber,ClaimNumber,ClaimFilingIndicator,PatientLastName,PatientFirstname,PatientIDCode,ServiceDate,PayerClaimControlNumber,LineCount,ServiceRevenueCode,ServiceCode,ServiceChargeAmount,ServicePaymentAmount,PatientAmount,ServiceAllowable,Modifier1,Modifier2,Reason Codes,Mastercode,subcode
0,OXFORD HEALTH INSURANCE INC,THP DALLAS HILLCREST MC LLC,2020-02-27,9802187,HILLF7563-1,HM,Raghavendra,Sai,1296444603,2020-01-12,0030H02069,1,300,85027,207.16,306.70,1203.19,1509.89,25,NaN,[45] Charge exceeds fee schedule/maximum allow...,80050,85027
1,OXFORD HEALTH INSURANCE INC,THP DALLAS HILLCREST MC LLC,2020-02-27,9802187,HILLF7563-1,HM,Raghavendra,Sai,1296444603,2020-01-12,0030H02069,2,300,85009,245.21,0.00,NaN,0.00,25,NaN,[23] The impact of prior payer(s) adjudication...,80050,85009
2,OXFORD HEALTH INSURANCE INC,THP DALLAS HILLCREST MC LLC,2020-02-27,9802187,HILLF7563-1,HM,Raghavendra,Sai,1296444603,2020-01-12,0030H02069,3,300,87340,245.21,0.00,NaN,0.00,25,NaN,[23] The impact of prior payer(s) adjudication...,80074,87340
3,OXFORD HEALTH INSURANCE INC,THP DALLAS HILLCREST MC LLC,2020-02-28,9812345,HILLF8471-2,HM,Raghavendra,Sai,981140604,2020-01-23,0042H01324,1,300,87340,245.21,0.00,1551.15,1551.15,25,NaN,[45] Charge exceeds fee schedule/maximum allow...,80074,87340
4,OXFORD HEALTH INSURANCE INC,THP DALLAS HILLCREST MC LLC,2020-02-28,9812345,HILLF8471-2,HM,Raghavendra,Sai,981140604,2020-01-23,0042H01324,2,300,86803,245.21,0.00,NaN,0.00,25,NaN,[23] The impact of prior payer(s) adjudication...,80074,86803
5,OXFORD HEALTH INSURANCE INC,THP DALLAS HILLCREST MC LLC,2020-03-26,10012508,HILLF9062-1,HM,Ramanathan,Prashanth,57282835003,2020-01-23,AY56462108,1,300,82330,207.16,0.00,NaN,0.00,QW,NaN,[242] Services not provided by network/primary...,80047,82330
6,OXFORD HEALTH INSURANCE INC,THP DALLAS HILLCREST MC LLC,2020-04-08,10089443,HILLF9092-1,HM,Ramanathan,Prashanth,1217690401,2020-01-26,0069H00806,1,300,82947,207.16,1256.83,300.00,1556.83,25,NaN,[45] Charge exceeds fee schedule/maximum allow...,80047,82947
7,OXFORD HEALTH INSURANCE INC,THP DALLAS HILLCREST MC LLC,2020-04-08,10089443,HILLF9092-1,HM,Ramanathan,Prashanth,1217690401,2020-01-26,0069H00806,2,300,80047,245.21,0.00,NaN,0.00,25,NaN,[23] The impact of prior payer(s) adjudication...,80047,80047
8,OXFORD HEALTH INSURANCE INC,THP DALLAS HILLCREST MC LLC,2020-04-08,10089443,HILLF9092-1,HM,G,Nidhi,1217690401,2020-01-26,0069H00806,3,300,84295,245.21,0.00,NaN,0.00,25,NaN,[23] The impact of prior payer(s) adjudication...,80047,84295
9,OXFORD HEALTH INSURANCE INC,THP DALLAS HILLCREST MC LLC,2020-04-10,10122647,HILLF3032-2,HM,G,Nidhi,1227260604,2020-02-25,0080H02390,1,300,84520,245.21,2134.33,533.58,2667.91,25,NaN,[45] Charge exceeds fee schedule/maximum allow...,80047,84520


In [39]:
from google.colab import files
patient_details.to_csv('2check.csv') 
files.download('2check.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>